In [ ]:
# # Here's the completion of the `counterfactual_reasoning_evaluation.py` file:

# # ```python
# import json
# import re
# import os
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from collections import defaultdict
# from tqdm import tqdm
# import torch
# import random
# from pathlib import Path
# from typing import List, Dict, Tuple, Any, Optional
# import copy

# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# # --- DATA LOADING ---
# def load_menatqa_dataset(file_path='./MenatQA.json'):
#     file_path = Path(file_path)
    
#     if not file_path.exists():
#         import urllib.request
#         print(f"Downloading MenatQA to {file_path}...")
#         urllib.request.urlretrieve(
#             "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
#             str(file_path)
#         )
    
#     with open(file_path, 'r') as f:
#         data = json.load(f)
#     print(f"Loaded MenatQA dataset with {len(data)} examples")
#     return data

# def extract_reasoning_hops(example):
#     question = example.get('question', '')
#     answer = example.get('answer', '')
#     q_type = example.get('type', '')
#     time_scope = example.get('time_scope', '')
    
#     sentences = [s.strip() for s in question.split('.') if s.strip()]
#     clauses = len([c for c in re.split(r'and|or|but|because|when|if', question.lower()) if c.strip()])
#     capitalized_words = len([w for w in question.split() if w and w[0].isupper()])
    
#     complexity_score = 1
#     complexity_score += min(1, len(sentences) - 1)
#     complexity_score += min(1, (clauses - 1) // 2)
#     complexity_score += min(1, capitalized_words // 3)
#     if time_scope:
#         complexity_score += 1
#     complexity_score = min(4, max(1, complexity_score))
    
#     return complexity_score

# def preprocess_dataset(data):
#     processed_data = []
#     hop_counts = {1: 0, 2: 0, 3: 0, 4: 0}
    
#     for item in data:
#         complexity_score = extract_reasoning_hops(item)
#         hop_counts[min(4, complexity_score)] += 1
        
#         entry = {
#             'ID': item.get('ID', ''),
#             'question': item.get('question', ''),
#             'answer': item.get('answer', ''),
#             'type': item.get('type', ''),
#             'time_scope': item.get('time_scope', ''),
#             'hop_count': complexity_score,
#         }
#         processed_data.append(entry)
    
#     df = pd.DataFrame(processed_data)
#     df['hop_category'] = pd.cut(
#         df['hop_count'],
#         bins=[-1, 1, 2, 3, float('inf')],
#         labels=['1-hop', '2-hop', '3-hop', '4+-hop']
#     )
    
#     print(f"Dataset preprocessed: {len(df)} questions")
#     for i in range(1, 5):
#         print(f"{i}-hop questions: {hop_counts[i]} ({hop_counts[i]/len(df):.1%})")
    
#     return df

# # --- COUNTERFACTUAL GENERATION ---
# def extract_modifiable_elements(question: str) -> Dict[str, List[Tuple[str, str]]]:
#     """Extract elements that can be modified for counterfactual reasoning"""
#     elements = {
#         'years': [],
#         'numbers': [],
#         'entities': [],
#         'locations': [],
#         'time_expressions': []
#     }
    
#     # Extract years (4-digit numbers that look like years)
#     year_pattern = r'\b(1\d{3}|20\d{2})\b'
#     years = re.findall(year_pattern, question)
#     for year in years:
#         elements['years'].append((year, f"year_{year}"))
    
#     # Extract other numbers
#     number_pattern = r'\b(\d+(?:\.\d+)?)\b'
#     numbers = re.findall(number_pattern, question)
#     for num in numbers:
#         if num not in years:  # Don't duplicate years
#             elements['numbers'].append((num, f"number_{num}"))
    
#     # Extract proper nouns/entities (capitalized words)
#     entity_pattern = r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b'
#     entities = re.findall(entity_pattern, question)
#     for entity in entities:
#         if not re.match(year_pattern, entity):  # Don't include years as entities
#             elements['entities'].append((entity, f"entity_{entity.replace(' ', '_')}"))
    
#     # Extract time expressions
#     time_patterns = [
#         r'\b(before|after|during|in|at|on)\s+(\d{4}|\w+\s+\d{4})\b',
#         r'\b(early|late|mid)[\s-](\d{4}s?)\b',
#         r'\b(\d{4})[\s-](to|until|through)[\s-](\d{4})\b'
#     ]
    
#     for pattern in time_patterns:
#         matches = re.findall(pattern, question, re.IGNORECASE)
#         for match in matches:
#             if isinstance(match, tuple):
#                 time_expr = ' '.join(match)
#             else:
#                 time_expr = match
#             elements['time_expressions'].append((time_expr, f"time_{time_expr.replace(' ', '_')}"))
    
#     # Extract locations (common location indicators)
#     location_indicators = ['in', 'at', 'from', 'to', 'near', 'around']
#     location_pattern = r'\b(?:' + '|'.join(location_indicators) + r')\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b'
#     locations = re.findall(location_pattern, question)
#     for location in locations:
#         elements['locations'].append((location, f"location_{location.replace(' ', '_')}"))
    
#     return elements

# def generate_counterfactual_changes(elements: Dict[str, List[Tuple[str, str]]]) -> List[Dict[str, Any]]:
#     """Generate different types of counterfactual changes"""
#     changes = []
    
#     # Year modifications (+/- 1-10 years)
#     for original_year, identifier in elements['years']:
#         year_val = int(original_year)
#         modifications = [
#             (str(year_val + 1), f"year_plus_1"),
#             (str(year_val - 1), f"year_minus_1"),
#             (str(year_val + 5), f"year_plus_5"),
#             (str(year_val - 5), f"year_minus_5"),
#             (str(year_val + 10), f"year_plus_10"),
#         ]
        
#         for new_year, change_type in modifications:
#             if 1800 <= int(new_year) <= 2030:  # Reasonable year range
#                 changes.append({
#                     'type': 'year_change',
#                     'original_value': original_year,
#                     'new_value': new_year,
#                     'change_description': change_type,
#                     'element_type': 'year'
#                 })
    
#     # Number modifications (+/- percentage)
#     for original_num, identifier in elements['numbers']:
#         try:
#             num_val = float(original_num)
#             modifications = [
#                 (str(int(num_val * 1.1)), f"number_plus_10_percent"),
#                 (str(int(num_val * 0.9)), f"number_minus_10_percent"),
#                 (str(int(num_val * 1.2)), f"number_plus_20_percent"),
#                 (str(int(num_val + 1)), f"number_plus_1"),
#                 (str(max(1, int(num_val - 1))), f"number_minus_1"),
#             ]
            
#             for new_num, change_type in modifications:
#                 changes.append({
#                     'type': 'number_change',
#                     'original_value': original_num,
#                     'new_value': new_num,
#                     'change_description': change_type,
#                     'element_type': 'number'
#                 })
#         except ValueError:
#             continue
    
#     # Entity substitutions (hypothetical alternatives)
#     entity_substitutions = {
#         'John': 'Peter', 'Mary': 'Sarah', 'Smith': 'Johnson', 'Brown': 'Wilson',
#         'United States': 'Canada', 'England': 'France', 'Germany': 'Italy',
#         'Company': 'Corporation', 'University': 'College', 'School': 'Academy'
#     }
    
#     for original_entity, identifier in elements['entities']:
#         # Direct substitutions
#         if original_entity in entity_substitutions:
#             new_entity = entity_substitutions[original_entity]
#             changes.append({
#                 'type': 'entity_substitution',
#                 'original_value': original_entity,
#                 'new_value': new_entity,
#                 'change_description': f"substitute_{original_entity}_with_{new_entity}",
#                 'element_type': 'entity'
#             })
        
#         # Generic substitutions
#         if any(word in original_entity.lower() for word in ['president', 'king', 'queen', 'leader']):
#             changes.append({
#                 'type': 'entity_substitution',
#                 'original_value': original_entity,
#                 'new_value': 'Prime Minister',
#                 'change_description': f"substitute_leader",
#                 'element_type': 'entity'
#             })
    
#     return changes

# def apply_counterfactual_change(question: str, change: Dict[str, Any]) -> str:
#     """Apply a counterfactual change to the question"""
#     modified_question = question
    
#     # Replace the original value with the new value
#     original_value = change['original_value']
#     new_value = change['new_value']
    
#     # Use word boundaries for exact matches
#     pattern = r'\b' + re.escape(original_value) + r'\b'
#     modified_question = re.sub(pattern, new_value, modified_question)
    
#     return modified_question

# # --- MODEL GENERATION ---
# def setup_model(model_name="Qwen/Qwen3-0.6B"):
#     """Initialize the model and tokenizer"""
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     print(f"Loading {model_name} on {device}...")
    
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
    
#     pipe = pipeline(
#         "text-generation", 
#         model=model, 
#         tokenizer=tokenizer, 
#         device=0 if device == 'cuda' else -1,
#         max_new_tokens=250,
#         temperature=0.01,
#         do_sample=True
#     )
    
#     return pipe

# def format_counterfactual_prompt(original_question: str, modified_question: str, change_description: str) -> str:
#     """Format prompt for counterfactual reasoning"""
#     return (f"Given this change in the question, please provide step-by-step reasoning to find the new answer.\n\n"
#             f"Original Question: {original_question}\n\n"
#             f"Modified Question (changed: {change_description}): {modified_question}\n\n"
#             f"Let's think step by step about how this change affects the answer:\n"
#             f"Step 1:")

# def extract_reasoning_steps(generation: str, prompt: str) -> List[str]:
#     """Extract reasoning steps from model generation"""
#     # Remove the original prompt
#     output = generation[len(prompt):] if generation.startswith(prompt) else generation
    
#     # Split by step indicators
#     steps = []
#     step_pattern = r'Step \d+:'
#     step_splits = re.split(step_pattern, output)
    
#     for i, step_content in enumerate(step_splits[1:], 1):  # Skip first empty split
#         step_text = step_content.strip().split('\n')[0].strip()  # Get first line of each step
#         if step_text:
#             steps.append(f"Step {i}: {step_text}")
    
#     # If no steps found, try to extract logical flow
#     if not steps:
#         lines = [line.strip() for line in output.split('\n') if line.strip()]
#         steps = lines[:5]  # Take first 5 meaningful lines
    
#     return steps

# def extract_final_answer(generation: str, prompt: str) -> str:
#     """Extract the final answer from generation"""
#     output = generation[len(prompt):] if generation.startswith(prompt) else generation
    
#     # Look for answer patterns
#     answer_patterns = [
#         r'(?:final answer|answer|conclusion)[\s:]*(.+?)(?:\n|$)',
#         r'(?:therefore|thus|so)[\s,]*(.+?)(?:\n|$)',
#         r'(?:the answer is|answer is)[\s:]*(.+?)(?:\n|$)',
#     ]
    
#     for pattern in answer_patterns:
#         match = re.search(pattern, output, re.IGNORECASE)
#         if match:
#             answer = match.group(1).strip()
#             # Clean answer
#             answer = re.sub(r'[.!?]*$', '', answer).strip()
#             if answer:
#                 return answer
    
#     # Fallback: look for the last meaningful line
#     lines = [line.strip() for line in output.split('\n') if line.strip()]
#     if lines:
#         return lines[-1]
    
#     return "No answer found"

# # --- COUNTERFACTUAL EVALUATION ---
# def evaluate_change_incorporation(original_steps: List[str], modified_steps: List[str],
#                                 change: Dict[str, Any]) -> Dict[str, Any]:
#     """Evaluate how well the model incorporated the counterfactual change"""
    
#     # Check if the change is mentioned in the reasoning
#     change_mentioned = False
#     original_value = change['original_value'].lower()
#     new_value = change['new_value'].lower()
    
#     modified_text = ' '.join(modified_steps).lower()
    
#     # Check if new value is mentioned
#     if new_value in modified_text:
#         change_mentioned = True
    
#     # Check if the model explicitly acknowledges the change
#     change_indicators = ['changed', 'modified', 'different', 'instead', 'now', 'updated']
#     change_acknowledged = any(indicator in modified_text for indicator in change_indicators)
    
#     # Check logical consistency - steps should be different if change is significant
#     original_text = ' '.join(original_steps).lower()
    
#     # Calculate similarity between reasoning chains
#     original_words = set(original_text.split())
#     modified_words = set(modified_text.split())
    
#     if len(original_words) > 0:
#         similarity = len(original_words & modified_words) / len(original_words | modified_words)
#     else:
#         similarity = 0
    
#     # Evaluate adaptation quality
#     adaptation_score = 0
#     if change_mentioned:
#         adaptation_score += 2
#     if change_acknowledged:
#         adaptation_score += 2
#     if similarity < 0.8:  # Reasoning changed significantly
#         adaptation_score += 1
    
#     # Check for step-by-step logical flow
#     logical_flow_score = 0
#     if len(modified_steps) >= 2:
#         for i in range(len(modified_steps) - 1):
#             current_step = modified_steps[i].lower()
#             next_step = modified_steps[i + 1].lower()
#             # Check for logical connectors
#             if any(conn in next_step for conn in ['because', 'therefore', 'since', 'so', 'thus']):
#                 logical_flow_score += 1
    
#     return {
#         'change_mentioned': change_mentioned,
#         'change_acknowledged': change_acknowledged,
#         'reasoning_similarity': similarity,
#         'adaptation_score': min(5, adaptation_score),
#         'logical_flow_score': logical_flow_score,
#         'reasoning_steps_count': len(modified_steps)
#     }

# def evaluate_reasoning_consistency(steps: List[str]) -> Dict[str, Any]:
#     """Evaluate the consistency and quality of reasoning steps"""
    
#     if not steps:
#         return {
#             'has_steps': False,
#             'step_count': 0,
#             'logical_connectors': 0,
#             'consistency_score': 0
#         }
    
#     # Count logical connectors
#     connector_words = ['because', 'therefore', 'since', 'so', 'thus', 'hence', 'consequently', 'as a result']
#     logical_connectors = 0
    
#     for step in steps:
#         step_lower = step.lower()
#         for connector in connector_words:
#             if connector in step_lower:
#                 logical_connectors += 1
#                 break
    
#     # Check for contradictions (simple heuristic)
#     contradiction_indicators = ['but', 'however', 'although', 'despite', 'unless', 'except']
#     contradictions = 0
    
#     for step in steps:
#         step_lower = step.lower()
#         for indicator in contradiction_indicators:
#             if indicator in step_lower:
#                 contradictions += 1
#                 break
    
#     # Calculate consistency score (higher is better)
#     consistency_score = len(steps)  # Base score for having steps
#     consistency_score += logical_connectors * 2  # Bonus for logical flow
#     consistency_score -= contradictions  # Penalty for contradictions
#     consistency_score = max(0, consistency_score)  # Ensure non-negative
    
#     return {
#         'has_steps': True,
#         'step_count': len(steps),
#         'logical_connectors': logical_connectors,
#         'contradictions': contradictions,
#         'consistency_score': consistency_score
#     }

# def calculate_counterfactual_score(evaluation_results: Dict[str, Any]) -> float:
#     """Calculate overall counterfactual reasoning score"""
    
#     # Weights for different aspects
#     weights = {
#         'adaptation': 0.4,      # How well the model adapted to the change
#         'consistency': 0.3,     # Internal consistency of reasoning
#         'logical_flow': 0.2,    # Logical flow between steps
#         'completeness': 0.1     # Completeness of reasoning
#     }
    
#     # Normalize scores to 0-1 range
#     adaptation_norm = evaluation_results.get('adaptation_score', 0) / 5.0
#     consistency_norm = min(1.0, evaluation_results.get('consistency_score', 0) / 10.0)
#     logical_flow_norm = min(1.0, evaluation_results.get('logical_flow_score', 0) / 3.0)
#     completeness_norm = min(1.0, evaluation_results.get('reasoning_steps_count', 0) / 5.0)
    
#     # Calculate weighted score
#     total_score = (
#         weights['adaptation'] * adaptation_norm +
#         weights['consistency'] * consistency_norm +
#         weights['logical_flow'] * logical_flow_norm +
#         weights['completeness'] * completeness_norm
#     )
    
#     return total_score

# # --- EVALUATION PIPELINE ---
# def run_counterfactual_evaluation(pipe, df: pd.DataFrame, num_samples: int = 100) -> Dict[str, Any]:
#     """Run full counterfactual reasoning evaluation"""
    
#     results = []
#     sample_data = df.sample(n=min(num_samples, len(df)), random_state=42)
    
#     print(f"Running counterfactual evaluation on {len(sample_data)} examples...")
    
#     for idx, row in tqdm(sample_data.iterrows(), total=len(sample_data)):
#         try:
#             # Extract modifiable elements
#             elements = extract_modifiable_elements(row['question'])
            
#             # Skip if no modifiable elements found
#             if not any(elements.values()):
#                 continue
            
#             # Generate counterfactual changes
#             changes = generate_counterfactual_changes(elements)
            
#             if not changes:
#                 continue
            
#             # Select one random change
#             change = random.choice(changes)
            
#             # Apply counterfactual change
#             modified_question = apply_counterfactual_change(row['question'], change)
            
#             # Generate reasoning for original question
#             original_prompt = f"Question: {row['question']}\nLet's think step by step:\nStep 1:"
#             original_generation = pipe(original_prompt, max_new_tokens=200)[0]['generated_text']
#             original_steps = extract_reasoning_steps(original_generation, original_prompt)
#             original_answer = extract_final_answer(original_generation, original_prompt)
            
#             # Generate reasoning for modified question
#             counterfactual_prompt = format_counterfactual_prompt(
#                 row['question'], modified_question, change['change_description']
#             )
#             modified_generation = pipe(counterfactual_prompt, max_new_tokens=200)[0]['generated_text']
#             modified_steps = extract_reasoning_steps(modified_generation, counterfactual_prompt)
#             modified_answer = extract_final_answer(modified_generation, counterfactual_prompt)
            
#             # Evaluate change incorporation
#             change_eval = evaluate_change_incorporation(original_steps, modified_steps, change)
            
#             # Evaluate reasoning consistency
#             consistency_eval = evaluate_reasoning_consistency(modified_steps)
            
#             # Combine evaluations
#             combined_eval = {**change_eval, **consistency_eval}
            
#             # Calculate overall score
#             overall_score = calculate_counterfactual_score(combined_eval)
            
#             result = {
#                 'id': row['ID'],
#                 'hop_category': row['hop_category'],
#                 'original_question': row['question'],
#                 'modified_question': modified_question,
#                 'change_type': change['type'],
#                 'change_description': change['change_description'],
#                 'original_steps': original_steps,
#                 'modified_steps': modified_steps,
#                 'original_answer': original_answer,
#                 'modified_answer': modified_answer,
#                 'evaluation': combined_eval,
#                 'overall_score': overall_score
#             }
            
#             results.append(result)
            
#         except Exception as e:
#             print(f"Error processing example {idx}: {e}")
#             continue
    
#     return results

# # --- ANALYSIS AND VISUALIZATION ---
# def analyze_results(results: List[Dict[str, Any]]) -> Dict[str, Any]:
#     """Analyze counterfactual reasoning results"""
    
#     if not results:
#         return {"error": "No results to analyze"}
    
#     # Convert to DataFrame for easier analysis
#     df_results = pd.DataFrame(results)
    
#     # Overall statistics
#     overall_stats = {
#         'total_examples': len(results),
#         'average_score': df_results['overall_score'].mean(),
#         'score_std': df_results['overall_score'].std(),
#         'change_mentioned_rate': df_results['evaluation'].apply(lambda x: x['change_mentioned']).mean(),
#         'change_acknowledged_rate': df_results['evaluation'].apply(lambda x: x['change_acknowledged']).mean(),
#         'avg_adaptation_score': df_results['evaluation'].apply(lambda x: x['adaptation_score']).mean(),
#         'avg_consistency_score': df_results['evaluation'].apply(lambda x: x['consistency_score']).mean()
#     }
    
#     # Analysis by hop category
#     hop_analysis = {}
#     for hop_cat in df_results['hop_category'].unique():
#         hop_data = df_results[df_results['hop_category'] == hop_cat]
#         hop_analysis[hop_cat] = {
#             'count': len(hop_data),
#             'avg_score': hop_data['overall_score'].mean(),
#             'change_mentioned_rate': hop_data['evaluation'].apply(lambda x: x['change_mentioned']).mean(),
#             'avg_steps': hop_data['evaluation'].apply(lambda x: x['reasoning_steps_count']).mean()
#         }
    
#     # Analysis by change type
#     change_analysis = {}
#     for change_type in df_results['change_type'].unique():
#         change_data = df_results[df_results['change_type'] == change_type]
#         change_analysis[change_type] = {
#             'count': len(change_data),
#             'avg_score': change_data['overall_score'].mean(),
#             'success_rate': change_data['evaluation'].apply(lambda x: x['adaptation_score'] >= 3).mean()
#         }
    
#     return {
#         'overall_stats': overall_stats,
#         'hop_analysis': hop_analysis,
#         'change_analysis': change_analysis,
#         'detailed_results': results
#     }

# def create_visualizations(analysis_results: Dict[str, Any], save_dir: str = "./plots"):
#     """Create visualizations for counterfactual reasoning results"""
    
#     os.makedirs(save_dir, exist_ok=True)
    
#     # Overall score distribution
#     plt.figure(figsize=(10, 6))
#     scores = [r['overall_score'] for r in analysis_results['detailed_results']]
#     plt.hist(scores, bins=20, alpha=0.7, edgecolor='black')
#     plt.xlabel('Overall Counterfactual Score')
#     plt.ylabel('Frequency')
#     plt.title('Distribution of Counterfactual Reasoning Scores')
#     plt.axvline(np.mean(scores), color='red', linestyle='--', label=f'Mean: {np.mean(scores):.3f}')
#     plt.legend()
#     plt.tight_layout()
#     plt.savefig(f"{save_dir}/score_distribution.png", dpi=300, bbox_inches='tight')
#     plt.close()
    
#     # Performance by hop category
#     if 'hop_analysis' in analysis_results:
#         hop_data = analysis_results['hop_analysis']
#         hop_categories = list(hop_data.keys())
#         hop_scores = [hop_data[cat]['avg_score'] for cat in hop_categories]
        
#         plt.figure(figsize=(10, 6))
#         plt.bar(hop_categories, hop_scores, alpha=0.7, color='skyblue', edgecolor='black')
#         plt.xlabel('Reasoning Complexity (Hop Category)')
#         plt.ylabel('Average Counterfactual Score')
#         plt.title('Counterfactual Reasoning Performance by Complexity')
#         plt.xticks(rotation=45)
#         plt.tight_layout()
#         plt.savefig(f"{save_dir}/performance_by_hops.png", dpi=300, bbox_inches='tight')
#         plt.close()
    
#     # Performance by change type
#     if 'change_analysis' in analysis_results:
#         change_data = analysis_results['change_analysis']
#         change_types = list(change_data.keys())
#         change_scores = [change_data[ct]['avg_score'] for ct in change_types]
        
#         plt.figure(figsize=(12, 6))
#         plt.bar(change_types, change_scores, alpha=0.7, color='lightcoral', edgecolor='black')
#         plt.xlabel('Change Type')
#         plt.ylabel('Average Counterfactual Score')
#         plt.title('Counterfactual Reasoning Performance by Change Type')
#         plt.xticks(rotation=45)
#         plt.tight_layout()
#         plt.savefig(f"{save_dir}/performance_by_change_type.png", dpi=300, bbox_inches='tight')
#         plt.close()

# def save_detailed_results(results: List[Dict[str, Any]], analysis: Dict[str, Any], 
#                          filename: str = "counterfactual_results.json"):
#     """Save detailed results to JSON file"""
    
#     output_data = {
#         'analysis_summary': {k: v for k, v in analysis.items() if k != 'detailed_results'},
#         'detailed_results': results,
#         'metadata': {
#             'num_examples': len(results),
#             'timestamp': pd.Timestamp.now().isoformat(),
#             'evaluation_type': 'counterfactual_reasoning'
#         }
#     }
    
#     with open(filename, 'w') as f:
#         json.dump(output_data, f, indent=2, default=str)
    
#     print(f"Results saved to {filename}")

# # --- MAIN EXECUTION ---
# def main():
#     """Main execution function"""
    
#     # Configuration
#     MODEL_NAME = "Qwen/Qwen3-0.6B"  # You can change this to other models
#     NUM_SAMPLES = 50  # Number of examples to evaluate
#     SAVE_DIR = "./counterfactual_evaluation_results"
    
#     os.makedirs(SAVE_DIR, exist_ok=True)
    
#     print("=== Counterfactual Reasoning Evaluation ===")
    
#     # Load and preprocess dataset
#     print("\n1. Loading and preprocessing dataset...")
#     data = load_menatqa_dataset()
#     df = preprocess_dataset(data)
    
#     # Setup model
#     print("\n2. Setting up model...")
#     pipe = setup_model(MODEL_NAME)
    
#     # Run evaluation
#     print("\n3. Running counterfactual evaluation...")
#     results = run_counterfactual_evaluation(pipe, df, num_samples=NUM_SAMPLES)
    
#     if not results:
#         print("No results generated. Please check your data and model setup.")
#         return
    
#     # Analyze results
#     print("\n4. Analyzing results...")
#     analysis = analyze_results(results)
    
#     # Print summary
#     print("\n=== EVALUATION SUMMARY ===")
#     if 'overall_stats' in analysis:
#         stats = analysis['overall_stats']
#         print(f"Total examples evaluated: {stats['total_examples']}")
#         print(f"Average counterfactual score: {stats['average_score']:.3f} ± {stats['score_std']:.3f}")
#         print(f"Change mentioned rate: {stats['change_mentioned_rate']:.1%}")
#         print(f"Change acknowledged rate: {stats['change_acknowledged_rate']:.1%}")
#         print(f"Average adaptation score: {stats['avg_adaptation_score']:.2f}/5")
#         print(f"Average consistency score: {stats['avg_consistency_score']:.2f}")
    
#     # Create visualizations
#     print("\n5. Creating visualizations...")
#     create_visualizations(analysis, f"{SAVE_DIR}/plots")
    
#     # Save results
#     print("\n6. Saving results...")
#     save_detailed_results(results, analysis, f"{SAVE_DIR}/detailed_results.json")
    
#     print(f"\nEvaluation complete! Results saved in {SAVE_DIR}/")
#     print(f"Check the plots in {SAVE_DIR}/plots/ for visualizations.")

# if __name__ == "__main__":
#     main()

=== Counterfactual Reasoning Evaluation ===

1. Loading and preprocessing dataset...
Loaded MenatQA dataset with 999 examples
Dataset preprocessed: 999 questions
1-hop questions: 0 (0.0%)
2-hop questions: 135 (13.5%)
3-hop questions: 698 (69.9%)
4-hop questions: 166 (16.6%)

2. Setting up model...
Loading Qwen/Qwen3-0.6B on cuda...


Device set to use cuda:0



3. Running counterfactual evaluation...
Running counterfactual evaluation on 50 examples...


 64%|██████▍   | 32/50 [06:57<03:48, 12.70s/it]

In [ ]:
import json
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
import torch
import random
from pathlib import Path
from typing import List, Dict, Tuple, Any, Optional
import copy

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# --- DATA LOADING ---
def load_menatqa_dataset(file_path='./MenatQA.json'):
    file_path = Path(file_path)
    
    if not file_path.exists():
        import urllib.request
        print(f"Downloading MenatQA to {file_path}...")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/weiyifan1023/MenatQA/main/datasets/MenatQA.json",
            str(file_path)
        )
    
    with open(file_path, 'r') as f:
        data = json.load(f)
    print(f"Loaded MenatQA dataset with {len(data)} examples")
    return data

def extract_reasoning_hops(example):
    question = example.get('question', '')
    answer = example.get('answer', '')
    q_type = example.get('type', '')
    time_scope = example.get('time_scope', '')
    
    sentences = [s.strip() for s in question.split('.') if s.strip()]
    clauses = len([c for c in re.split(r'and|or|but|because|when|if', question.lower()) if c.strip()])
    capitalized_words = len([w for w in question.split() if w and w[0].isupper()])
    
    complexity_score = 1
    complexity_score += min(1, len(sentences) - 1)
    complexity_score += min(1, (clauses - 1) // 2)
    complexity_score += min(1, capitalized_words // 3)
    if time_scope:
        complexity_score += 1
    complexity_score = min(4, max(1, complexity_score))
    
    return complexity_score

def preprocess_dataset(data):
    processed_data = []
    hop_counts = {1: 0, 2: 0, 3: 0, 4: 0}
    
    for item in data:
        complexity_score = extract_reasoning_hops(item)
        hop_counts[min(4, complexity_score)] += 1
        
        entry = {
            'ID': item.get('ID', ''),
            'question': item.get('question', ''),
            'answer': item.get('answer', ''),
            'type': item.get('type', ''),
            'time_scope': item.get('time_scope', ''),
            'hop_count': complexity_score,
        }
        processed_data.append(entry)
    
    df = pd.DataFrame(processed_data)
    df['hop_category'] = pd.cut(
        df['hop_count'],
        bins=[-1, 1, 2, 3, float('inf')],
        labels=['1-hop', '2-hop', '3-hop', '4+-hop']
    )
    
    print(f"Dataset preprocessed: {len(df)} questions")
    for i in range(1, 5):
        print(f"{i}-hop questions: {hop_counts[i]} ({hop_counts[i]/len(df):.1%})")
    
    return df

# --- COUNTERFACTUAL GENERATION ---
def extract_modifiable_elements(question: str) -> Dict[str, List[Tuple[str, str]]]:
    """Extract elements that can be modified for counterfactual reasoning"""
    elements = {
        'years': [],
        'numbers': [],
        'entities': [],
        'locations': [],
        'time_expressions': []
    }
    
    # Extract years (4-digit numbers that look like years)
    year_pattern = r'\b(1\d{3}|20\d{2})\b'
    years = re.findall(year_pattern, question)
    for year in years:
        elements['years'].append((year, f"year_{year}"))
    
    # Extract other numbers
    number_pattern = r'\b(\d+(?:\.\d+)?)\b'
    numbers = re.findall(number_pattern, question)
    for num in numbers:
        if num not in years:  # Don't duplicate years
            elements['numbers'].append((num, f"number_{num}"))
    
    # Extract proper nouns/entities (capitalized words)
    entity_pattern = r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b'
    entities = re.findall(entity_pattern, question)
    for entity in entities:
        if not re.match(year_pattern, entity):  # Don't include years as entities
            elements['entities'].append((entity, f"entity_{entity.replace(' ', '_')}"))
    
    # Extract time expressions
    time_patterns = [
        r'\b(before|after|during|in|at|on)\s+(\d{4}|\w+\s+\d{4})\b',
        r'\b(early|late|mid)[\s-](\d{4}s?)\b',
        r'\b(\d{4})[\s-](to|until|through)[\s-](\d{4})\b'
    ]
    
    for pattern in time_patterns:
        matches = re.findall(pattern, question, re.IGNORECASE)
        for match in matches:
            if isinstance(match, tuple):
                time_expr = ' '.join(match)
            else:
                time_expr = match
            elements['time_expressions'].append((time_expr, f"time_{time_expr.replace(' ', '_')}"))
    
    # Extract locations (common location indicators)
    location_indicators = ['in', 'at', 'from', 'to', 'near', 'around']
    location_pattern = r'\b(?:' + '|'.join(location_indicators) + r')\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b'
    locations = re.findall(location_pattern, question)
    for location in locations:
        elements['locations'].append((location, f"location_{location.replace(' ', '_')}"))
    
    return elements

def generate_counterfactual_changes(elements: Dict[str, List[Tuple[str, str]]]) -> List[Dict[str, Any]]:
    """Generate different types of counterfactual changes"""
    changes = []
    
    # Year modifications (+/- 1-10 years)
    for original_year, identifier in elements['years']:
        year_val = int(original_year)
        modifications = [
            (str(year_val + 1), f"year_plus_1"),
            (str(year_val - 1), f"year_minus_1"),
            (str(year_val + 5), f"year_plus_5"),
            (str(year_val - 5), f"year_minus_5"),
            (str(year_val + 10), f"year_plus_10"),
        ]
        
        for new_year, change_type in modifications:
            if 1800 <= int(new_year) <= 2030:  # Reasonable year range
                changes.append({
                    'type': 'year_change',
                    'original_value': original_year,
                    'new_value': new_year,
                    'change_description': change_type,
                    'element_type': 'year'
                })
    
    # Number modifications (+/- percentage)
    for original_num, identifier in elements['numbers']:
        try:
            num_val = float(original_num)
            modifications = [
                (str(int(num_val * 1.1)), f"number_plus_10_percent"),
                (str(int(num_val * 0.9)), f"number_minus_10_percent"),
                (str(int(num_val * 1.2)), f"number_plus_20_percent"),
                (str(int(num_val + 1)), f"number_plus_1"),
                (str(max(1, int(num_val - 1))), f"number_minus_1"),
            ]
            
            for new_num, change_type in modifications:
                changes.append({
                    'type': 'number_change',
                    'original_value': original_num,
                    'new_value': new_num,
                    'change_description': change_type,
                    'element_type': 'number'
                })
        except ValueError:
            continue
    
    # Entity substitutions (hypothetical alternatives)
    entity_substitutions = {
        'John': 'Peter', 'Mary': 'Sarah', 'Smith': 'Johnson', 'Brown': 'Wilson',
        'United States': 'Canada', 'England': 'France', 'Germany': 'Italy',
        'Company': 'Corporation', 'University': 'College', 'School': 'Academy'
    }
    
    for original_entity, identifier in elements['entities']:
        # Direct substitutions
        if original_entity in entity_substitutions:
            new_entity = entity_substitutions[original_entity]
            changes.append({
                'type': 'entity_substitution',
                'original_value': original_entity,
                'new_value': new_entity,
                'change_description': f"substitute_{original_entity}_with_{new_entity}",
                'element_type': 'entity'
            })
        
        # Generic substitutions
        if any(word in original_entity.lower() for word in ['president', 'king', 'queen', 'leader']):
            changes.append({
                'type': 'entity_substitution',
                'original_value': original_entity,
                'new_value': 'Prime Minister',
                'change_description': f"substitute_leader",
                'element_type': 'entity'
            })
    
    return changes

def apply_counterfactual_change(question: str, change: Dict[str, Any]) -> str:
    """Apply a counterfactual change to the question"""
    modified_question = question
    
    # Replace the original value with the new value
    original_value = change['original_value']
    new_value = change['new_value']
    
    # Use word boundaries for exact matches
    pattern = r'\b' + re.escape(original_value) + r'\b'
    modified_question = re.sub(pattern, new_value, modified_question)
    
    return modified_question

# --- MODEL GENERATION ---
def setup_model(model_name="Qwen/Qwen3-0.6B"):
    """Initialize the model and tokenizer"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading {model_name} on {device}...")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
    
    pipe = pipeline(
        "text-generation", 
        model=model, 
        tokenizer=tokenizer, 
        device=0 if device == 'cuda' else -1,
        max_new_tokens=250,
        temperature=0.01,
        do_sample=True
    )
    
    return pipe

def format_counterfactual_prompt(original_question: str, modified_question: str, change_description: str) -> str:
    """Format prompt for counterfactual reasoning"""
    return (f"Given this change in the question, please provide step-by-step reasoning to find the new answer.\n\n"
            f"Original Question: {original_question}\n\n"
            f"Modified Question (changed: {change_description}): {modified_question}\n\n"
            f"Let's think step by step about how this change affects the answer:\n"
            f"Step 1:")

def extract_reasoning_steps(generation: str, prompt: str) -> List[str]:
    """Extract reasoning steps from model generation"""
    # Remove the original prompt
    output = generation[len(prompt):] if generation.startswith(prompt) else generation
    
    # Split by step indicators
    steps = []
    step_pattern = r'Step \d+:'
    step_splits = re.split(step_pattern, output)
    
    for i, step_content in enumerate(step_splits[1:], 1):  # Skip first empty split
        step_text = step_content.strip().split('\n')[0].strip()  # Get first line of each step
        if step_text:
            steps.append(f"Step {i}: {step_text}")
    
    # If no steps found, try to extract logical flow
    if not steps:
        lines = [line.strip() for line in output.split('\n') if line.strip()]
        steps = lines[:5]  # Take first 5 meaningful lines
    
    return steps

def extract_final_answer(generation: str, prompt: str) -> str:
    """Extract the final answer from generation"""
    output = generation[len(prompt):] if generation.startswith(prompt) else generation
    
    # Look for answer patterns
    answer_patterns = [
        r'(?:final answer|answer|conclusion)[\s:]*(.+?)(?:\n|$)',
        r'(?:therefore|thus|so)[\s,]*(.+?)(?:\n|$)',
        r'(?:the answer is|answer is)[\s:]*(.+?)(?:\n|$)',
    ]
    
    for pattern in answer_patterns:
        match = re.search(pattern, output, re.IGNORECASE)
        if match:
            answer = match.group(1).strip()
            # Clean answer
            answer = re.sub(r'[.!?]*$', '', answer).strip()
            if answer:
                return answer
    
    # Fallback: look for the last meaningful line
    lines = [line.strip() for line in output.split('\n') if line.strip()]
    if lines:
        return lines[-1]
    
    return "No answer found"

# --- COUNTERFACTUAL EVALUATION ---
def evaluate_change_incorporation(original_steps: List[str], modified_steps: List[str],
                                change: Dict[str, Any]) -> Dict[str, Any]:
    """Evaluate how well the model incorporated the counterfactual change"""
    
    # Check if the change is mentioned in the reasoning
    change_mentioned = False
    original_value = change['original_value'].lower()
    new_value = change['new_value'].lower()
    
    modified_text = ' '.join(modified_steps).lower()
    
    # Check if new value is mentioned
    if new_value in modified_text:
        change_mentioned = True
    
    # Check if the model explicitly acknowledges the change
    change_indicators = ['changed', 'modified', 'different', 'instead', 'now', 'updated']
    change_acknowledged = any(indicator in modified_text for indicator in change_indicators)
    
    # Check logical consistency - steps should be different if change is significant
    original_text = ' '.join(original_steps).lower()
    
    # Calculate similarity between reasoning chains
    original_words = set(original_text.split())
    modified_words = set(modified_text.split())
    
    if len(original_words) > 0:
        similarity = len(original_words & modified_words) / len(original_words | modified_words)
    else:
        similarity = 0
    
    # Evaluate adaptation quality
    adaptation_score = 0
    if change_mentioned:
        adaptation_score += 2
    if change_acknowledged:
        adaptation_score += 2
    if similarity < 0.8:  # Reasoning changed significantly
        adaptation_score += 1
    
    # Check for step-by-step logical flow
    logical_flow_score = 0
    if len(modified_steps) >= 2:
        for i in range(len(modified_steps) - 1):
            current_step = modified_steps[i].lower()
            next_step = modified_steps[i + 1].lower()
            # Check for logical connectors
            if any(conn in next_step for conn in ['because', 'therefore', 'since', 'so', 'thus']):
                logical_flow_score += 1
    
    return {
        'change_mentioned': change_mentioned,
        'change_acknowledged': change_acknowledged,
        'reasoning_similarity': similarity,
        'adaptation_score': min(5, adaptation_score),
        'logical_flow_score': logical_flow_score,
        'reasoning_steps_count': len(modified_steps)
    }

def evaluate_reasoning_consistency(steps: List[str]) -> Dict[str, Any]:
    """Evaluate the consistency and quality of reasoning steps"""
    
    if not steps:
        return {
            'has_steps': False,
            'step_count': 0,
            'logical_connectors': 0,
            'consistency_score': 0
        }
    
    # Count logical connectors
    connector_words = ['because', 'therefore', 'since', 'so', 'thus', 'hence', 'consequently', 'as a result']
    logical_connectors = 0
    
    for step in steps:
        step_lower = step.lower()
        for connector in connector_words:
            if connector in step_lower:
                logical_connectors += 1
                break
    
    # Check for contradictions (simple heuristic)
    contradiction_indicators = ['but', 'however', 'although', 'despite', 'unless', 'except']
    contradictions = 0
    
    for step in steps:
        step_lower = step.lower()
        for indicator in contradiction_indicators:
            if indicator in step_lower:
                contradictions += 1
                break
    
    # Calculate consistency score (higher is better)
    consistency_score = len(steps)  # Base score for having steps
    consistency_score += logical_connectors * 2  # Bonus for logical flow
    consistency_score -= contradictions  # Penalty for contradictions
    consistency_score = max(0, consistency_score)  # Ensure non-negative
    
    return {
        'has_steps': True,
        'step_count': len(steps),
        'logical_connectors': logical_connectors,
        'contradictions': contradictions,
        'consistency_score': consistency_score
    }

def calculate_counterfactual_score(evaluation_results: Dict[str, Any]) -> float:
    """Calculate overall counterfactual reasoning score"""
    
    # Weights for different aspects
    weights = {
        'adaptation': 0.4,      # How well the model adapted to the change
        'consistency': 0.3,     # Internal consistency of reasoning
        'logical_flow': 0.2,    # Logical flow between steps
        'completeness': 0.1     # Completeness of reasoning
    }
    
    # Normalize scores to 0-1 range
    adaptation_norm = evaluation_results.get('adaptation_score', 0) / 5.0
    consistency_norm = min(1.0, evaluation_results.get('consistency_score', 0) / 10.0)
    logical_flow_norm = min(1.0, evaluation_results.get('logical_flow_score', 0) / 3.0)
    completeness_norm = min(1.0, evaluation_results.get('reasoning_steps_count', 0) / 5.0)
    
    # Calculate weighted score
    total_score = (
        weights['adaptation'] * adaptation_norm +
        weights['consistency'] * consistency_norm +
        weights['logical_flow'] * logical_flow_norm +
        weights['completeness'] * completeness_norm
    )
    
    return total_score

# --- EVALUATION PIPELINE ---
def run_counterfactual_evaluation(pipe, df: pd.DataFrame, batch_size: int = 10) -> Dict[str, Any]:
    """Run full counterfactual reasoning evaluation on ALL data"""
    
    results = []
    processed_count = 0
    failed_count = 0
    
    print(f"Running counterfactual evaluation on ALL {len(df)} examples...")
    print(f"Processing in batches of {batch_size} for memory efficiency...")
    
    # Process all data in batches
    for start_idx in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
        end_idx = min(start_idx + batch_size, len(df))
        batch_data = df.iloc[start_idx:end_idx]
        
        for idx, row in batch_data.iterrows():
            try:
                # Extract modifiable elements
                elements = extract_modifiable_elements(row['question'])
                
                # Skip if no modifiable elements found
                if not any(elements.values()):
                    failed_count += 1
                    continue
                
                # Generate counterfactual changes
                changes = generate_counterfactual_changes(elements)
                
                if not changes:
                    failed_count += 1
                    continue
                
                # Select one random change (or you could process all changes)
                change = random.choice(changes)
                
                # Apply counterfactual change
                modified_question = apply_counterfactual_change(row['question'], change)
                
                # Generate reasoning for original question
                original_prompt = f"Question: {row['question']}\nLet's think step by step:\nStep 1:"
                original_generation = pipe(original_prompt, max_new_tokens=200)[0]['generated_text']
                original_steps = extract_reasoning_steps(original_generation, original_prompt)
                original_answer = extract_final_answer(original_generation, original_prompt)
                
                # Generate reasoning for modified question
                counterfactual_prompt = format_counterfactual_prompt(
                    row['question'], modified_question, change['change_description']
                )
                modified_generation = pipe(counterfactual_prompt, max_new_tokens=200)[0]['generated_text']
                modified_steps = extract_reasoning_steps(modified_generation, counterfactual_prompt)
                modified_answer = extract_final_answer(modified_generation, counterfactual_prompt)
                
                # Evaluate change incorporation
                change_eval = evaluate_change_incorporation(original_steps, modified_steps, change)
                
                # Evaluate reasoning consistency
                consistency_eval = evaluate_reasoning_consistency(modified_steps)
                
                # Combine evaluations
                combined_eval = {**change_eval, **consistency_eval}
                
                # Calculate overall score
                overall_score = calculate_counterfactual_score(combined_eval)
                
                result = {
                    'id': row['ID'],
                    'hop_category': row['hop_category'],
                    'original_question': row['question'],
                    'modified_question': modified_question,
                    'change_type': change['type'],
                    'change_description': change['change_description'],
                    'original_steps': original_steps,
                    'modified_steps': modified_steps,
                    'original_answer': original_answer,
                    'modified_answer': modified_answer,
                    'evaluation': combined_eval,
                    'overall_score': overall_score
                }
                
                results.append(result)
                processed_count += 1
                
                # Progress update every 100 examples
                if processed_count % 100 == 0:
                    print(f"Processed: {processed_count}, Failed: {failed_count}, Success rate: {processed_count/(processed_count+failed_count):.1%}")
                
                # Optional: Clear CUDA cache periodically to prevent memory issues
                if torch.cuda.is_available() and processed_count % 50 == 0:
                    torch.cuda.empty_cache()
                
            except Exception as e:
                print(f"Error processing example {idx}: {e}")
                failed_count += 1
                continue
    
    print(f"\nEvaluation completed!")
    print(f"Successfully processed: {processed_count}")
    print(f"Failed: {failed_count}")
    print(f"Total success rate: {processed_count/(processed_count+failed_count):.1%}")
    
    return results

# --- ANALYSIS AND VISUALIZATION ---
def analyze_results(results: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Analyze counterfactual reasoning results"""
    
    if not results:
        return {"error": "No results to analyze"}
    
    # Convert to DataFrame for easier analysis
    df_results = pd.DataFrame(results)
    
    # Overall statistics
    overall_stats = {
        'total_examples': len(results),
        'average_score': df_results['overall_score'].mean(),
        'score_std': df_results['overall_score'].std(),
        'median_score': df_results['overall_score'].median(),
        'min_score': df_results['overall_score'].min(),
        'max_score': df_results['overall_score'].max(),
        'change_mentioned_rate': df_results['evaluation'].apply(lambda x: x['change_mentioned']).mean(),
        'change_acknowledged_rate': df_results['evaluation'].apply(lambda x: x['change_acknowledged']).mean(),
        'avg_adaptation_score': df_results['evaluation'].apply(lambda x: x['adaptation_score']).mean(),
        'avg_consistency_score': df_results['evaluation'].apply(lambda x: x['consistency_score']).mean(),
        'avg_logical_flow_score': df_results['evaluation'].apply(lambda x: x['logical_flow_score']).mean(),
        'avg_reasoning_steps': df_results['evaluation'].apply(lambda x: x['reasoning_steps_count']).mean()
    }
    
    # Analysis by hop category
    hop_analysis = {}
    for hop_cat in df_results['hop_category'].unique():
        hop_data = df_results[df_results['hop_category'] == hop_cat]
        hop_analysis[hop_cat] = {
            'count': len(hop_data),
            'avg_score': hop_data['overall_score'].mean(),
            'std_score': hop_data['overall_score'].std(),
            'change_mentioned_rate': hop_data['evaluation'].apply(lambda x: x['change_mentioned']).mean(),
            'change_acknowledged_rate': hop_data['evaluation'].apply(lambda x: x['change_acknowledged']).mean(),
            'avg_adaptation_score': hop_data['evaluation'].apply(lambda x: x['adaptation_score']).mean(),
            'avg_steps': hop_data['evaluation'].apply(lambda x: x['reasoning_steps_count']).mean()
        }
    
    # Analysis by change type
    change_analysis = {}
    for change_type in df_results['change_type'].unique():
        change_data = df_results[df_results['change_type'] == change_type]
        change_analysis[change_type] = {
            'count': len(change_data),
            'avg_score': change_data['overall_score'].mean(),
            'std_score': change_data['overall_score'].std(),
            'success_rate': change_data['evaluation'].apply(lambda x: x['adaptation_score'] >= 3).mean(),
            'change_mentioned_rate': change_data['evaluation'].apply(lambda x: x['change_mentioned']).mean()
        }
    
    # Performance distribution analysis
    score_ranges = {
        'excellent': (0.8, 1.0),
        'good': (0.6, 0.8),
        'fair': (0.4, 0.6),
        'poor': (0.0, 0.4)
    }
    
    performance_distribution = {}
    for range_name, (min_score, max_score) in score_ranges.items():
        count = len(df_results[(df_results['overall_score'] >= min_score) & 
                              (df_results['overall_score'] < max_score)])
        performance_distribution[range_name] = {
            'count': count,
            'percentage': count / len(df_results) * 100
        }
    
    return {
        'overall_stats': overall_stats,
        'hop_analysis': hop_analysis,
        'change_analysis': change_analysis,
        'performance_distribution': performance_distribution,
        'detailed_results': results
    }

def create_comprehensive_visualizations(analysis_results: Dict[str, Any], save_dir: str = "./plots"):
    """Create comprehensive visualizations for counterfactual reasoning results"""
    
    os.makedirs(save_dir, exist_ok=True)
    
    # 1. Overall score distribution
    plt.figure(figsize=(12, 8))
    scores = [r['overall_score'] for r in analysis_results['detailed_results']]
    
    plt.subplot(2, 2, 1)
    plt.hist(scores, bins=30, alpha=0.7, edgecolor='black', color='skyblue')
    plt.xlabel('Overall Counterfactual Score')
    plt.ylabel('Frequency')
    plt.title('Distribution of Counterfactual Reasoning Scores')
    plt.axvline(np.mean(scores), color='red', linestyle='--', label=f'Mean: {np.mean(scores):.3f}')
    plt.axvline(np.median(scores), color='orange', linestyle='--', label=f'Median: {np.median(scores):.3f}')
    plt.legend()
    
    # 2. Performance by hop category
    if 'hop_analysis' in analysis_results:
        hop_data = analysis_results['hop_analysis']
        hop_categories = list(hop_data.keys())
        hop_scores = [hop_data[cat]['avg_score'] for cat in hop_categories]
        hop_stds = [hop_data[cat]['std_score'] for cat in hop_categories]
        
        plt.subplot(2, 2, 2)
        bars = plt.bar(hop_categories, hop_scores, alpha=0.7, color='lightcoral', 
                      edgecolor='black', yerr=hop_stds, capsize=5)
        plt.xlabel('Reasoning Complexity (Hop Category)')
        plt.ylabel('Average Counterfactual Score')
        plt.title('Performance by Reasoning Complexity')
        plt.xticks(rotation=45)
        
        # Add count labels on bars
        for i, (bar, cat) in enumerate(zip(bars, hop_categories)):
            count = hop_data[cat]['count']
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'n={count}', ha='center', va='bottom', fontsize=8)
    
    # 3. Performance by change type
    if 'change_analysis' in analysis_results:
        change_data = analysis_results['change_analysis']
        change_types = list(change_data.keys())
        change_scores = [change_data[ct]['avg_score'] for ct in change_types]
        change_stds = [change_data[ct]['std_score'] for ct in change_types]
        
        plt.subplot(2, 2, 3)
        bars = plt.bar(change_types, change_scores, alpha=0.7, color='lightgreen', 
                      edgecolor='black', yerr=change_stds, capsize=5)
        plt.xlabel('Change Type')
        plt.ylabel('Average Counterfactual Score')
        plt.title('Performance by Change Type')
        plt.xticks(rotation=45)
        
        # Add count labels on bars
        for i, (bar, ct) in enumerate(zip(bars, change_types)):
            count = change_data[ct]['count']
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'n={count}', ha='center', va='bottom', fontsize=8)
    
    # 4. Performance distribution pie chart
    if 'performance_distribution' in analysis_results:
        perf_data = analysis_results['performance_distribution']
        labels = list(perf_data.keys())
        sizes = [perf_data[label]['percentage'] for label in labels]
        colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
        
        plt.subplot(2, 2, 4)
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        plt.title('Performance Distribution')
    
    plt.tight_layout()
    plt.savefig(f"{save_dir}/comprehensive_analysis.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 5. Detailed adaptation metrics
    plt.figure(figsize=(15, 5))
    
    # Change mentioned vs acknowledged rates by hop category
    if 'hop_analysis' in analysis_results:
        hop_data = analysis_results['hop_analysis']
        hop_categories = list(hop_data.keys())
        mentioned_rates = [hop_data[cat]['change_mentioned_rate'] for cat in hop_categories]
        acknowledged_rates = [hop_data[cat]['change_acknowledged_rate'] for cat in hop_categories]
        
        x = np.arange(len(hop_categories))
        width = 0.35
        
        plt.subplot(1, 3, 1)
        plt.bar(x - width/2, mentioned_rates, width, label='Change Mentioned', alpha=0.7)
        plt.bar(x + width/2, acknowledged_rates, width, label='Change Acknowledged', alpha=0.7)
        plt.xlabel('Hop Category')
        plt.ylabel('Rate')
        plt.title('Change Recognition by Complexity')
        plt.xticks(x, hop_categories, rotation=45)
        plt.legend()
        plt.ylim(0, 1)
    
    # Adaptation scores distribution
    adaptation_scores = [r['evaluation']['adaptation_score'] for r in analysis_results['detailed_results']]
    plt.subplot(1, 3, 2)
    plt.hist(adaptation_scores, bins=6, alpha=0.7, edgecolor='black', color='purple')
    plt.xlabel('Adaptation Score (0-5)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Adaptation Scores')
    plt.xticks(range(6))
    
    # Reasoning steps vs performance correlation
    reasoning_steps = [r['evaluation']['reasoning_steps_count'] for r in analysis_results['detailed_results']]
    overall_scores = [r['overall_score'] for r in analysis_results['detailed_results']]
    
    plt.subplot(1, 3, 3)
    plt.scatter(reasoning_steps, overall_scores, alpha=0.6, color='orange')
    plt.xlabel('Number of Reasoning Steps')
    plt.ylabel('Overall Score')
    plt.title('Reasoning Steps vs Performance')
    
    # Add correlation coefficient
    correlation = np.corrcoef(reasoning_steps, overall_scores)[0, 1]
    plt.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
             transform=plt.gca().transAxes, bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(f"{save_dir}/detailed_metrics.png", dpi=300, bbox_inches='tight')
    plt.close()

def save_comprehensive_results(results: List[Dict[str, Any]], analysis: Dict[str, Any], 
                              filename: str = "counterfactual_results_complete.json"):
    """Save comprehensive results to JSON file"""
    
    output_data = {
        'analysis_summary': {k: v for k, v in analysis.items() if k != 'detailed_results'},
        'detailed_results': results,
        'metadata': {
            'num_examples': len(results),
            'timestamp': pd.Timestamp.now().isoformat(),
            'evaluation_type': 'counterfactual_reasoning_complete_dataset',
            'total_dataset_size': 'all_available_data'
        }
    }
    
    with open(filename, 'w') as f:
        json.dump(output_data, f, indent=2, default=str)
    
    print(f"Complete results saved to {filename}")
    
    # Also save a summary CSV for easy analysis
    summary_filename = filename.replace('.json', '_summary.csv')
    summary_data = []
    
    for result in results:
        summary_row = {
            'id': result['id'],
            'hop_category': result['hop_category'],
            'change_type': result['change_type'],
            'overall_score': result['overall_score'],
            'change_mentioned': result['evaluation']['change_mentioned'],
            'change_acknowledged': result['evaluation']['change_acknowledged'],
            'adaptation_score': result['evaluation']['adaptation_score'],
            'consistency_score': result['evaluation']['consistency_score'],
            'logical_flow_score': result['evaluation']['logical_flow_score'],
            'reasoning_steps_count': result['evaluation']['reasoning_steps_count'],
            'reasoning_similarity': result['evaluation']['reasoning_similarity']
        }
        summary_data.append(summary_row)
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(summary_filename, index=False)
    print(f"Summary CSV saved to {summary_filename}")

def print_detailed_summary(analysis: Dict[str, Any]):
    """Print a detailed summary of the evaluation results"""
    
    print("\n" + "="*80)
    print("COMPREHENSIVE COUNTERFACTUAL REASONING EVALUATION RESULTS")
    print("="*80)
    
    if 'overall_stats' in analysis:
        stats = analysis['overall_stats']
        print(f"\n📊 OVERALL STATISTICS:")
        print(f"   Total examples evaluated: {stats['total_examples']:,}")
        print(f"   Average counterfactual score: {stats['average_score']:.3f} ± {stats['score_std']:.3f}")
        print(f"   Median score: {stats['median_score']:.3f}")
        print(f"   Score range: {stats['min_score']:.3f} - {stats['max_score']:.3f}")
        print(f"   Change mentioned rate: {stats['change_mentioned_rate']:.1%}")
        print(f"   Change acknowledged rate: {stats['change_acknowledged_rate']:.1%}")
        print(f"   Average adaptation score: {stats['avg_adaptation_score']:.2f}/5")
        print(f"   Average consistency score: {stats['avg_consistency_score']:.2f}")
        print(f"   Average logical flow score: {stats['avg_logical_flow_score']:.2f}")
        print(f"   Average reasoning steps: {stats['avg_reasoning_steps']:.1f}")
    
    if 'performance_distribution' in analysis:
        print(f"\n🎯 PERFORMANCE DISTRIBUTION:")
        perf_dist = analysis['performance_distribution']
        for level, data in perf_dist.items():
            print(f"   {level.capitalize()}: {data['count']:,} examples ({data['percentage']:.1f}%)")
    
    if 'hop_analysis' in analysis:
        print(f"\n🔢 PERFORMANCE BY REASONING COMPLEXITY:")
        hop_analysis = analysis['hop_analysis']
        for hop_cat, data in hop_analysis.items():
            print(f"   {hop_cat}:")
            print(f"     Count: {data['count']:,}")
            print(f"     Avg Score: {data['avg_score']:.3f} ± {data['std_score']:.3f}")
            print(f"     Change Recognition: {data['change_mentioned_rate']:.1%}")
            print(f"     Change Acknowledgment: {data['change_acknowledged_rate']:.1%}")
            print(f"     Avg Steps: {data['avg_steps']:.1f}")
    
    if 'change_analysis' in analysis:
        print(f"\n🔄 PERFORMANCE BY CHANGE TYPE:")
        change_analysis = analysis['change_analysis']
        for change_type, data in change_analysis.items():
            print(f"   {change_type}:")
            print(f"     Count: {data['count']:,}")
            print(f"     Avg Score: {data['avg_score']:.3f} ± {data['std_score']:.3f}")
            print(f"     Success Rate: {data['success_rate']:.1%}")
            print(f"     Change Recognition: {data['change_mentioned_rate']:.1%}")

def create_final_report(analysis: Dict[str, Any], save_dir: str):
    """Create a final evaluation report"""
    
    report_filename = f"{save_dir}/evaluation_report.txt"
    
    with open(report_filename, 'w') as f:
        f.write("COUNTERFACTUAL REASONING EVALUATION REPORT\n")
        f.write("="*50 + "\n\n")
        f.write(f"Generated on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        if 'overall_stats' in analysis:
            stats = analysis['overall_stats']
            f.write("EXECUTIVE SUMMARY\n")
            f.write("-" * 20 + "\n")
            f.write(f"Total examples evaluated: {stats['total_examples']:,}\n")
            f.write(f"Overall performance score: {stats['average_score']:.3f}/1.0\n")
            f.write(f"Change recognition rate: {stats['change_mentioned_rate']:.1%}\n")
            f.write(f"Change acknowledgment rate: {stats['change_acknowledged_rate']:.1%}\n\n")
            
            f.write("KEY FINDINGS\n")
            f.write("-" * 12 + "\n")
            
            # Performance assessment
            if stats['average_score'] >= 0.7:
                f.write("✅ STRONG: Model demonstrates strong counterfactual reasoning capabilities\n")
            elif stats['average_score'] >= 0.5:
                f.write("⚠️  MODERATE: Model shows moderate counterfactual reasoning capabilities\n")
            else:
                f.write("❌ WEAK: Model shows limited counterfactual reasoning capabilities\n")
            
            # Change recognition assessment
            if stats['change_mentioned_rate'] >= 0.7:
                f.write("✅ GOOD: Model effectively recognizes changes in questions\n")
            elif stats['change_mentioned_rate'] >= 0.4:
                f.write("⚠️  FAIR: Model moderately recognizes changes in questions\n")
            else:
                f.write("❌ POOR: Model struggles to recognize changes in questions\n")
            
            f.write("\nDETAILED STATISTICS\n")
            f.write("-" * 19 + "\n")
            for key, value in stats.items():
                if isinstance(value, float):
                    f.write(f"{key}: {value:.3f}\n")
                else:
                    f.write(f"{key}: {value}\n")
    
    print(f"Final report saved to {report_filename}")

# --- MAIN EXECUTION ---
def main():
    """Main execution function - processes ALL data"""
    
    # Configuration
    MODEL_NAME = "microsoft/DialoGPT-medium"  # You can change this to other models
    BATCH_SIZE = 20  # Process in batches for memory efficiency
    SAVE_DIR = "./counterfactual_evaluation_complete"
    
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    print("=== COMPREHENSIVE COUNTERFACTUAL REASONING EVALUATION ===")
    print("Processing ENTIRE dataset - this may take several hours depending on dataset size and hardware")
    
    # Load and preprocess dataset
    print("\n1. Loading and preprocessing dataset...")
    data = load_menatqa_dataset()
    df = preprocess_dataset(data)
    
    print(f"\n📋 Dataset Summary:")
    print(f"   Total questions: {len(df):,}")
    print(f"   Hop distribution:")
    for hop_cat in df['hop_category'].value_counts().sort_index().items():
        print(f"     {hop_cat[0]}: {hop_cat[1]:,} ({hop_cat[1]/len(df):.1%})")
    
    # Setup model
    print("\n2. Setting up model...")
    pipe = setup_model(MODEL_NAME)
    
    # Estimate processing time
    estimated_time_hours = len(df) * 0.5 / 60  # Rough estimate: 30 seconds per example
    print(f"\n⏱️  Estimated processing time: {estimated_time_hours:.1f} hours")
    
    response = input("\nDo you want to proceed with the full evaluation? (y/n): ")
    if response.lower() != 'y':
        print("Evaluation cancelled.")
        return
    
    # Run evaluation on ALL data
    print("\n3. Running comprehensive counterfactual evaluation...")
    print("💡 Tip: This will process the entire dataset. Consider running in screen/tmux for long sessions.")
    
    results = run_counterfactual_evaluation(pipe, df, batch_size=BATCH_SIZE)
    
    if not results:
        print("❌ No results generated. Please check your data and model setup.")
        return
    
    # Analyze results
    print("\n4. Analyzing comprehensive results...")
    analysis = analyze_results(results)
    
    # Print detailed summary
    print_detailed_summary(analysis)
    
    # Create comprehensive visualizations
    print("\n5. Creating comprehensive visualizations...")
    create_comprehensive_visualizations(analysis, f"{SAVE_DIR}/plots")
    
    # Save complete results
    print("\n6. Saving comprehensive results...")
    save_comprehensive_results(results, analysis, f"{SAVE_DIR}/complete_results.json")
    
    # Create final report
    print("\n7. Generating final report...")
    create_final_report(analysis, SAVE_DIR)
    
    print(f"\n🎉 COMPREHENSIVE EVALUATION COMPLETE!")
    print(f"📁 All results saved in: {SAVE_DIR}/")
    print(f"📊 Visualizations: {SAVE_DIR}/plots/")
    print(f"📄 Summary CSV: {SAVE_DIR}/complete_results_summary.csv")
    print(f"📋 Final report: {SAVE_DIR}/evaluation_report.txt")
    
    # Final statistics
    if 'overall_stats' in analysis:
        stats = analysis['overall_stats']
        print(f"\n📈 FINAL PERFORMANCE SUMMARY:")
        print(f"   Evaluated: {stats['total_examples']:,} examples")
        print(f"   Overall Score: {stats['average_score']:.3f}/1.0")
        print(f"   Success Rate: {stats['change_mentioned_rate']:.1%}")

if __name__ == "__main__":
    # Set random seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    
    main()